In [13]:
import pandas as pd
import numpy as np

In [14]:
df_base=pd.read_csv("test_data.csv")
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99888 entries, 0 to 99887
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           99888 non-null  object 
 1   customer_id              99888 non-null  object 
 2   age                      99888 non-null  int64  
 3   gender                   99888 non-null  object 
 4   marital_status           99888 non-null  object 
 5   employment_status        99888 non-null  object 
 6   income_bracket           99888 non-null  object 
 7   account_tenure_months    99888 non-null  int64  
 8   credit_limit             99888 non-null  int64  
 9   avg_monthly_spend        99888 non-null  float64
 10  card_type                99888 non-null  object 
 11  card_issuer              99888 non-null  object 
 12  has_addon_card           99888 non-null  object 
 13  is_blocked               99888 non-null  object 
 14  transaction_time      

In [15]:
# Risk Score Calculation between 0 to 1000

In [16]:
df = df_base.copy()

In [17]:
# Amount Bases Rules
# Ensure numeric inputs
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')
df['avg_monthly_spend'] = pd.to_numeric(df['avg_monthly_spend'], errors='coerce')
df['txn_amount_to_avg_ratio'] = pd.to_numeric(df['txn_amount_to_avg_ratio'], errors='coerce')
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')


In [18]:
# Precompute threshold
df['high_threshold'] = df['avg_monthly_spend'] * 0.1  # 3/30 = 0.1

# Thresholds and margin
reporting_thresholds = [50000, 200000, 1000000]
margin = 5000

# Initialize score
df['risk_high_amount_ratio'] = 0
df['risk_round_number'] = 0
df['risk_below_threshold'] = 0

#### Amount-Based Rules (Weight: 15-25 points)

In [19]:
# 1. "Amount > 95th percentile of customer history
df.loc[df['transaction_amount'] > df['high_threshold'], 'risk_high_amount_ratio'] = 25 

In [20]:
# 2. "Transaction-to-average ratio > 5x"

df.loc[df['txn_amount_to_avg_ratio'] > 5.0, 'risk_txn_to_avg_ratio'] = 25
df.loc[(df['txn_amount_to_avg_ratio'] > 2.0) & (df['txn_amount_to_avg_ratio'] <= 5.0), 'risk_txn_to_avg_ratio'] = 15
df.loc[(df['txn_amount_to_avg_ratio'] > 1.5) & (df['txn_amount_to_avg_ratio'] <= 2.0), 'risk_txn_to_avg_ratio'] = 5

In [21]:
# 3. "Round number amounts (ending in 00)"

df.loc[(df['transaction_amount'] % 100 == 0) & (df['transaction_amount'] >= 1000), 'risk_round_number'] = 10
df.loc[(df['transaction_amount'] % 1000 == 0) & (df['transaction_amount'] >= 5000), 'risk_round_number'] = 15

In [22]:
# 4. "Amount just below reporting threshold"
# This rule identifies transactions that are intentionally structured to fall just below
# regulatory or internal reporting thresholds.
# Define the thresholds and a small margin.
margin = 500 # Example margin
df['risk_below_threshold'] = 0 # Initialize with 0

# Vectorized approach to apply the rule efficiently
for threshold in reporting_thresholds:
    df.loc[(df['transaction_amount'] >= (threshold - margin)) & 
           (df['transaction_amount'] < threshold), 
           'risk_below_threshold'] = 15

#### Behavioral Pattern Rules (Weight: 20-35 points)

In [23]:
# Helper: parse time and derive hour if needed
if 'transaction_time' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['transaction_time']):
    df['transaction_time'] = pd.to_datetime(df['transaction_time'], errors='coerce')


In [24]:
# 1) First transaction on new device: +15
# Logic: a device is "new" for a customer if this is the first time the customer_id uses device_id historically.
# We flag when (customer_id, device_id) pair appears for the first time for that customer.
if {'customer_id','device_id'}.issubset(df.columns):
    df = df.sort_values(['customer_id','transaction_time','transaction_id'])
    first_seen_device = ~df.duplicated(subset=['customer_id','device_id'])
    df['risk_new_device_first_txn'] = first_seen_device.astype(int) * 15
else:
    df['risk_new_device_first_txn'] = 0

In [25]:
# 2) Device change during session: +25
# Approximate a session by consecutive transactions of the same customer within 30 minutes (configurable).
# A change is when device_id differs from the previous txn within-session.
SESSION_GAP_MIN = 30
if {'customer_id','transaction_time','device_id'}.issubset(df.columns):
    df = df.sort_values(['customer_id','transaction_time','transaction_id'])
    prev_time = df.groupby('customer_id')['transaction_time'].shift(1)
    prev_device = df.groupby('customer_id')['device_id'].shift(1)
    gap_min = (df['transaction_time'] - prev_time).dt.total_seconds() / 60
    same_session = gap_min.notna() & (gap_min <= SESSION_GAP_MIN)
    device_changed = same_session & (df['device_id'] != prev_device)
    df['risk_device_change_session'] = device_changed.astype(int) * 25
else:
    df['risk_device_change_session'] = 0

In [26]:
# 3) Unusual transaction hour (outside 6am-10pm): +20
# Define allowed window 06:00-22:00 inclusive of endpoints. Use transaction_hour if present; else derive from time.
if 'transaction_hour' in df.columns:
    hr = df['transaction_hour']
else:
    hr = df['transaction_time'].dt.hour
unusual_hour = (hr < 6) | (hr > 22)
df['risk_unusual_hour'] = unusual_hour.astype(int) * 20


In [27]:
# 4) Multiple transactions within 1 hour (>3): +30
# Add 30 points if customer has >3 transactions in the last hour (including current)
LOOKBACK_MIN = 60

if {'customer_id','transaction_time'}.issubset(df.columns):
    # CRITICAL: Convert transaction_time to datetime first
    df['transaction_time'] = pd.to_datetime(df['transaction_time'])
    
    # Sort by customer and time
    df = df.sort_values(['customer_id','transaction_time','transaction_id'])
    df = df.reset_index(drop=True)
    
    # Calculate total transaction counts in last hour (including current transaction)
    txn_counts_last_hour = []
    
    for idx, row in df.iterrows():
        current_customer = row['customer_id']
        current_time = row['transaction_time']
        
        # Get all transactions for this customer up to and including current transaction
        mask = (df.index <= idx) & (df['customer_id'] == current_customer)
        customer_transactions = df.loc[mask, 'transaction_time']
        
        # Count transactions in the last 60 minutes (including current)
        time_window = current_time - pd.Timedelta(minutes=LOOKBACK_MIN)
        count_in_last_hour = (customer_transactions >= time_window).sum()
        txn_counts_last_hour.append(count_in_last_hour)
    
    df['txns_last_hour_total'] = txn_counts_last_hour
    
    # Add 30 points if more than 3 transactions in last hour
    df['risk_multi_txns_1h'] = (df['txns_last_hour_total'] > 3).astype(int) * 30
else:
    df['risk_multi_txns_1h'] = 0


In [28]:
print("Sample results:")
print(df[['customer_id', 'transaction_time', 'txns_last_hour_total', 'risk_multi_txns_1h']].tail(10))

Sample results:
      customer_id    transaction_time  txns_last_hour_total  \
99878    CUST9999 2024-01-14 01:20:08                     1   
99879    CUST9999 2024-01-21 17:43:40                     1   
99880    CUST9999 2024-02-10 10:39:15                     1   
99881    CUST9999 2024-02-21 15:16:11                     1   
99882    CUST9999 2024-03-09 01:18:32                     1   
99883    CUST9999 2024-03-11 17:33:55                     1   
99884    CUST9999 2024-03-13 18:23:36                     1   
99885    CUST9999 2024-03-18 02:41:36                     1   
99886    CUST9999 2024-05-22 13:40:36                     1   
99887    CUST9999 2024-05-28 23:08:37                     1   

       risk_multi_txns_1h  
99878                   0  
99879                   0  
99880                   0  
99881                   0  
99882                   0  
99883                   0  
99884                   0  
99885                   0  
99886                   0  
99887     

In [29]:
# 1. Check basic statistics of txns_last_hour_total
print("=== Transaction Count Distribution ===")
print(df['txns_last_hour_total'].value_counts().sort_index())
print(f"\nMax transactions in 1 hour: {df['txns_last_hour_total'].max()}")
print(f"Customers with >1 transaction in last hour: {(df['txns_last_hour_total'] > 1).sum()}")
print(f"Customers with >3 transactions in last hour: {(df['txns_last_hour_total'] > 3).sum()}")

# 2. Show customers with multiple transactions (>1) in last hour
print("\n=== Customers with Multiple Transactions in Last Hour ===")
multiple_txns = df[df['txns_last_hour_total'] > 1].copy()
if len(multiple_txns) > 0:
    print(f"Found {len(multiple_txns)} transactions where customer had multiple txns in last hour")
    print(multiple_txns[['customer_id', 'transaction_time', 'txns_last_hour_total', 'risk_multi_txns_1h']].head(10))
else:
    print("No customers found with multiple transactions in last hour")

# 3. Show customers with >3 transactions (triggering the 30-point penalty)
print("\n=== Customers Triggering Risk Flag (>3 transactions) ===")
high_risk = df[df['txns_last_hour_total'] > 3].copy()
if len(high_risk) > 0:
    print(f"Found {len(high_risk)} transactions triggering 30-point risk penalty")
    print(high_risk[['customer_id', 'transaction_time', 'txns_last_hour_total', 'risk_multi_txns_1h']].head(10))
else:
    print("No customers found with >3 transactions in last hour")

# 4. Show specific customer examples with their transaction timeline
print("\n=== Customer Timeline Examples ===")
# Find customers who had multiple transactions
customers_with_multiple = df[df['txns_last_hour_total'] > 1]['customer_id'].unique()

if len(customers_with_multiple) > 0:
    # Show timeline for first few customers
    for customer in customers_with_multiple[:3]:  # Show first 3 customers
        print(f"\n--- Customer {customer} Transaction Timeline ---")
        customer_data = df[df['customer_id'] == customer].sort_values('transaction_time')
        print(customer_data[['transaction_time', 'txns_last_hour_total', 'risk_multi_txns_1h']].to_string(index=False))
else:
    print("No customers found with multiple transactions to show timeline")

# 5. Summary by risk score
print(f"\n=== Risk Score Summary ===")
print(f"Transactions with 0 risk points: {(df['risk_multi_txns_1h'] == 0).sum()}")
print(f"Transactions with 30 risk points: {(df['risk_multi_txns_1h'] == 30).sum()}")
print(f"Percentage flagged as high risk: {(df['risk_multi_txns_1h'] == 30).mean()*100:.1f}%")

# 6. Verification check - make sure logic is working
print(f"\n=== Logic Verification ===")
print("Checking if risk_multi_txns_1h correctly maps to txns_last_hour_total > 3:")
correct_mapping = ((df['txns_last_hour_total'] > 3) == (df['risk_multi_txns_1h'] == 30)).all()
print(f"Logic is correct: {correct_mapping}")

if not correct_mapping:
    print("ERROR: Risk scoring logic mismatch detected!")
    mismatched = df[(df['txns_last_hour_total'] > 3) != (df['risk_multi_txns_1h'] == 30)]
    print(mismatched[['customer_id', 'txns_last_hour_total', 'risk_multi_txns_1h']].head())

=== Transaction Count Distribution ===
txns_last_hour_total
1    99635
2      253
Name: count, dtype: int64

Max transactions in 1 hour: 2
Customers with >1 transaction in last hour: 253
Customers with >3 transactions in last hour: 0

=== Customers with Multiple Transactions in Last Hour ===
Found 253 transactions where customer had multiple txns in last hour
     customer_id    transaction_time  txns_last_hour_total  risk_multi_txns_1h
1374    CUST1127 2023-12-31 19:58:51                     2                   0
1700    CUST1155 2024-05-25 03:33:47                     2                   0
2556    CUST1232 2024-01-26 21:03:32                     2                   0
2559    CUST1232 2024-03-20 17:49:45                     2                   0
4136    CUST1372 2023-12-16 00:00:48                     2                   0
4333    CUST1391 2023-12-31 07:15:39                     2                   0
4338    CUST1391 2024-03-09 21:21:21                     2                   0
4615  

In [30]:
# 5) Geographic distance from home > 500km: +25
# The dataset provides distance_from_home_km; use that directly if available.
if 'distance_from_home_km' in df.columns:
    df['risk_far_from_home'] = (df['distance_from_home_km'] > 500).astype(int) * 25
else:
    df['risk_far_from_home'] = 0

In [31]:
# Behavioral component capped 20–35 points
df['Behavioral_Subscore'] = df[existing].sum(axis=1).clip(upper=35)

NameError: name 'existing' is not defined

In [ ]:
df.query('fraud_flag == 1')

#### Historical Risk Rules (Weight: 30-50 points)

In [ ]:
df['risk_prior_fraud'] = (df['prior_fraud_count'] > 0).astype(int) * 40

In [ ]:
# Assuming 'account_creation_date' exists or can be derived from 'account_tenure_months'
if 'account_tenure_months' in df.columns:
    # Convert months to days (approximately)
    account_age_days = df['account_tenure_months'] * 30
    df['risk_new_account'] = (account_age_days < 30).astype(int) * 20
elif 'account_creation_date' in df.columns:
    df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
    account_age_days = (df['transaction_time'] - df['account_creation_date']).dt.days
    df['risk_new_account'] = (account_age_days < 30).astype(int) * 20

In [ ]:
# Assuming 'is_device_blacklisted' column exists (from your dataset)
if 'is_device_blacklisted' in df.columns:
    # Convert Yes/No to boolean if needed
    blacklisted = df['is_device_blacklisted'].map({'Yes': True, 'No': False}).fillna(False)
    df['risk_blacklisted_device'] = blacklisted.astype(int) * 50

In [ ]:
# ----------------------------------------
# 3) Blacklisted device → +50
# Using is_device_blacklisted flag
# ----------------------------------------
if 'is_device_blacklisted' in df.columns:
    df['risk_blacklisted_device'] = (df['is_device_blacklisted'] == True).astype(int) * 50
else:
    df['risk_blacklisted_device'] = 0

In [ ]:
# Option A: If we have 'is_blocked' status and block/unblock dates
if 'last_unblock_date' in df.columns:
    df['last_unblock_date'] = pd.to_datetime(df['last_unblock_date'])
    days_since_unblock = (df['transaction_time'] - df['last_unblock_date']).dt.days
    recently_unblocked = (days_since_unblock <= 30) & (days_since_unblock >= 0)
    df['risk_recent_unblock'] = recently_unblocked.astype(int) * 35


#### Merchant & Location Rules (Weight: 10-30 points)

In [ ]:
#1. Foreign country transaction: +25 points
# Assumption that customer's home country is India and Transaction are done by customer are from India as well

print("1. FOREIGN COUNTRY TRANSACTION RULE")
print("Logic: Transactions outside India are higher risk")
print("Countries in dataset:", df['merchant_country'].value_counts())

# Basic foreign country rule
df['risk_foreign_country'] = (df['merchant_country'] != 'IN').astype(int) * 25

# Enhanced version with high-risk countries
high_risk_countries = ['CN', 'RU', 'NG', 'PK', 'AE', 'ZA']  # Based on common fraud patterns
df['risk_foreign_country_enhanced'] = np.where(
    df['merchant_country'].isin(high_risk_countries), 30,  # Higher risk countries
    np.where(df['merchant_country'] != 'IN', 25, 0)        # Other foreign countries
)

print(f"Foreign transactions: {(df['merchant_country'] != 'IN').sum()}")
print(f"Fraud rate in foreign transactions: {df[df['merchant_country'] != 'IN']['fraud_flag'].mean():.2%}")
print(f"Fraud rate in domestic transactions: {df[df['merchant_country'] == 'IN']['fraud_flag'].mean():.2%}")


In [ ]:
# Distribution Check Analysis

In [ ]:
df['merchant_country'].value_counts(normalize=True, dropna=False) * 100

In [ ]:
df['merchant_category_code'].value_counts(normalize=True, dropna=False) * 100

In [ ]:
df['transaction_channel'].value_counts(normalize=True, dropna=False) * 100

In [ ]:
# 2. High-risk merchant category: +20 points
print("\n2. HIGH-RISK MERCHANT CATEGORY RULE")
print("Logic: Certain merchant categories are more prone to fraud")
print("Merchant categories in dataset:", df['merchant_category_code'].value_counts())

# Analyze fraud rates by merchant category
fraud_by_category = df.groupby('merchant_category_code').agg({
    'fraud_flag': ['count', 'sum', 'mean']
}).round(3)
fraud_by_category.columns = ['total_txns', 'fraud_count', 'fraud_rate']
print("\nFraud rates by merchant category:")
print(fraud_by_category.sort_values('fraud_rate', ascending=False))

# Define high-risk categories based on analysis
# Typically: Cash advances (6011), ATM (6010), Money transfer (6012), Gambling (7995)
high_risk_categories = ['6011']  # Based on your data analysis
medium_risk_categories = ['5944', '5999', '5732']  # Higher fraud rates observed

df['risk_merchant_category'] = np.where(
    df['merchant_category_code'].isin(high_risk_categories), 25,
    np.where(df['merchant_category_code'].isin(medium_risk_categories), 15, 0)
)

In [ ]:
# 3. New merchant (first time): +15 points
print("\n3. NEW MERCHANT RULE")
print("Logic: First-time transactions with a merchant are riskier")

# Create merchant history for each customer
df_sorted = df.sort_values(['customer_id', 'transaction_time'])
df_sorted['is_new_merchant'] = ~df_sorted.duplicated(subset=['customer_id', 'merchant_name'])

# Count previous transactions with same merchant
df_sorted['merchant_transaction_count'] = df_sorted.groupby(['customer_id', 'merchant_name']).cumcount()

df['risk_new_merchant'] = np.where(df_sorted['merchant_transaction_count'] == 0, 15, 0)

new_merchant_fraud_rate = df[df['risk_new_merchant'] > 0]['fraud_flag'].mean()
existing_merchant_fraud_rate = df[df['risk_new_merchant'] == 0]['fraud_flag'].mean()
print(f"Fraud rate for new merchants: {new_merchant_fraud_rate:.2%}")
print(f"Fraud rate for existing merchants: {existing_merchant_fraud_rate:.2%}")

In [ ]:
# 4. Cash advance transactions: +30 points
print("\n4. CASH ADVANCE TRANSACTION RULE")
print("Logic: ATM and cash-related transactions have higher fraud risk")

# ATM transactions and specific merchant categories
cash_advance_conditions = (
    (df['transaction_channel'] == 'ATM') |
    (df['merchant_category_code'] == '6011')  # Financial institutions
)

df['risk_cash_advance'] = cash_advance_conditions.astype(int) * 30

cash_advance_fraud_rate = df[cash_advance_conditions]['fraud_flag'].mean()
non_cash_advance_fraud_rate = df[~cash_advance_conditions]['fraud_flag'].mean()
print(f"Fraud rate for cash advance transactions: {cash_advance_fraud_rate:.2%}")
print(f"Fraud rate for non-cash advance transactions: {non_cash_advance_fraud_rate:.2%}")

In [ ]:
# 5. ADDITIONAL MERCHANT & LOCATION RULES

# Distance from home rule: +20 points
print("\n5. DISTANCE FROM HOME RULE")
print("Logic: Transactions far from usual location are suspicious")

# Using the distance_from_home_km field
distance_threshold = df['distance_from_home_km'].quantile(0.9)  # Top 10% distances
df['risk_distance_from_home'] = (df['distance_from_home_km'] > distance_threshold).astype(int) * 20

print(f"Distance threshold (90th percentile): {distance_threshold:.1f} km")
far_from_home_fraud_rate = df[df['distance_from_home_km'] > distance_threshold]['fraud_flag'].mean()
near_home_fraud_rate = df[df['distance_from_home_km'] <= distance_threshold]['fraud_flag'].mean()
print(f"Fraud rate for distant transactions: {far_from_home_fraud_rate:.2%}")
print(f"Fraud rate for nearby transactions: {near_home_fraud_rate:.2%}")


In [ ]:
# 6. High-risk location rule: +15 points
print("\n6. HIGH-RISK LOCATION RULE")
print("Logic: Certain cities/locations have higher fraud rates")

fraud_by_city = df.groupby('location_city').agg({
    'fraud_flag': ['count', 'sum', 'mean']
}).round(3)
fraud_by_city.columns = ['total_txns', 'fraud_count', 'fraud_rate']
high_risk_cities = fraud_by_city[fraud_by_city['fraud_rate'] > df['fraud_flag'].mean() * 1.5].index.tolist()

df['risk_high_risk_location'] = df['location_city'].isin(high_risk_cities).astype(int) * 15


In [ ]:
# 7. Cross-border IP mismatch: +25 points
print("\n7. IP LOCATION MISMATCH RULE")
print("Logic: Transaction location doesn't match IP geolocation pattern")

# Simple heuristic: if merchant is foreign but transaction seems domestic
# This would require actual IP geolocation, but we can simulate
df['risk_ip_mismatch'] = (
    (df['merchant_country'] != 'IN') & 
    (df['distance_from_home_km'] < 100)  # Seems like local transaction
).astype(int) * 25


In [ ]:
# ===== COMBINE ALL MERCHANT & LOCATION RULES =====
merchant_location_columns = [
    'risk_foreign_country_enhanced',
    'risk_merchant_category', 
    'risk_new_merchant',
    'risk_cash_advance',
    'risk_distance_from_home',
    'risk_high_risk_location',
    'risk_ip_mismatch'
]

df['merchant_location_risk_score'] = df[merchant_location_columns].sum(axis=1)

print("\n" + "="*50)
print("MERCHANT & LOCATION RISK SCORE ANALYSIS")
print("="*50)

print(f"Mean merchant/location risk score: {df['merchant_location_risk_score'].mean():.1f}")
print(f"Max merchant/location risk score: {df['merchant_location_risk_score'].max()}")


In [ ]:
# Analyze risk score distribution for fraud vs non-fraud
fraud_risk_scores = df[df['fraud_flag'] == 1]['merchant_location_risk_score']
non_fraud_risk_scores = df[df['fraud_flag'] == 0]['merchant_location_risk_score']

print(f"\nFraud transactions - Mean risk score: {fraud_risk_scores.mean():.1f}")
print(f"Non-fraud transactions - Mean risk score: {non_fraud_risk_scores.mean():.1f}")

# Risk score buckets analysis
print("\nRisk Score Distribution:")
risk_buckets = pd.cut(df['merchant_location_risk_score'], 
                     bins=[0, 25, 50, 75, 100, float('inf')], 
                     labels=['0-25', '26-50', '51-75', '76-100', '100+'])

bucket_analysis = df.groupby(risk_buckets).agg({
    'fraud_flag': ['count', 'sum', 'mean']
}).round(3)
bucket_analysis.columns = ['total_txns', 'fraud_count', 'fraud_rate']
print(bucket_analysis)


In [ ]:
# ===== RULE EFFECTIVENESS METRICS =====
print("\n" + "="*50)
print("RULE EFFECTIVENESS ANALYSIS")
print("="*50)

def calculate_rule_effectiveness(df, rule_column, rule_name):
    """Calculate effectiveness metrics for a rule"""
    rule_triggered = df[rule_column] > 0
    
    # Basic metrics
    trigger_rate = rule_triggered.sum() / len(df)
    fraud_detection_rate = df[rule_triggered]['fraud_flag'].sum() / df['fraud_flag'].sum()
    precision = df[rule_triggered]['fraud_flag'].mean()
    
    print(f"\n{rule_name}:")
    print(f"  Trigger rate: {trigger_rate:.2%}")
    print(f"  Fraud detection rate: {fraud_detection_rate:.2%}")
    print(f"  Precision: {precision:.2%}")
    
    return {
        'trigger_rate': trigger_rate,
        'fraud_detection_rate': fraud_detection_rate,
        'precision': precision
    }

# Calculate effectiveness for each rule
rule_effectiveness = {}
for col, name in zip(merchant_location_columns, 
                    ['Foreign Country', 'High-Risk Category', 'New Merchant', 
                     'Cash Advance', 'Distance from Home', 'High-Risk Location', 'IP Mismatch']):
    rule_effectiveness[name] = calculate_rule_effectiveness(df, col, name)


In [ ]:
# df.to_csv("out_data.csv")

#### Technical Anomaly Rules (Weight: 15-25 points)

In [ ]:
# 1. No OTP used for high amount (Weight: +20 points)

# Calculate the 95th percentile of all transaction amounts to define "high amount"
high_amount_threshold = df['transaction_amount'].quantile(0.95)

# Create a new column 'risk_no_otp_high_amount' and assign 20 points
# if the transaction amount is high and otp_used is 'No'.
df['risk_no_otp_high_amount'] = (
    (df['transaction_amount'] > high_amount_threshold) & 
    (df['otp_used'] == 'No')
).astype(int) * 20

In [ ]:
df[df['risk_no_otp_high_amount']>0]!=False 

In [ ]:
# 2. Manual entry instead of chip/contactless (Weight: +15 points)

# Create a new column 'risk_manual_entry' and assign 15 points
# if the entry_mode is 'manual'.
df['risk_manual_entry'] = (df['entry_mode'] == 'manual').astype(int) * 15

In [33]:
# --- 3. High Transaction Velocity (+25 points) ---
# This rule identifies a sudden increase in a customer's transaction frequency,
# which can signal an account takeover.
# We use pre-calculated features: `txns_last_1hr` and `txns_last_24hr`.
print("\nApplying Rule: High Transaction Velocity...")
# These thresholds can be adjusted based on business knowledge and data analysis.
txns_1hr_threshold = 3
txns_24hr_threshold = 10
df['risk_velocity_rule'] = (
    (df['txns_last_1hr'] > txns_1hr_threshold) |
    (df['txns_last_24hr'] > txns_24hr_threshold)
).astype(int) * 25



Applying Rule: High Transaction Velocity...


### Final Risk Score Calculation

In [41]:
df["Total_Risk_Score"] = df[
    ["risk_high_amount_ratio",
     "risk_round_number", 
     "risk_below_threshold",
     "risk_txn_to_avg_ratio", 
     "risk_velocity_rule", 
     "risk_new_device_first_txn",	
     "risk_device_change_session",	
     "risk_unusual_hour",
     "risk_multi_txns_1h",	
     "risk_far_from_home"]
].sum(axis=1)

In [42]:
df

,transaction_id,customer_id,age,gender,marital_status,employment_status,income_bracket,account_tenure_months,credit_limit,avg_monthly_spend,...,risk_below_threshold,risk_txn_to_avg_ratio,risk_new_device_first_txn,risk_device_change_session,risk_unusual_hour,txns_last_hour_total,risk_multi_txns_1h,risk_far_from_home,risk_velocity_rule,Total_Risk_Score
0,2e60c930-dc76-4b18-a691-0edfda2c4155,CUST1000,58,M,Married,Self-employed,>1L,120,200000,26917.50,...,0,15.0,15,0,20,1,0,25,0,100.0
1,84b6a9ae-3996-4cb0-b16a-6fa883b0ce1b,CUST1000,46,F,Single,Salaried,<25K,116,50000,44563.84,...,0,NaN,15,0,20,1,0,0,0,60.0
2,638d8f7e-a668-48ce-b635-0c6572f3d100,CUST1000,34,M,Divorced,Student,<25K,27,100000,45204.19,...,0,5.0,15,0,20,1,0,0,0,65.0
3,38d174bf-2f13-4af4-bc37-8bbb4ee782a3,CUST1000,46,M,Married,Unemployed,50K-1L,65,100000,44373.27,...,0,15.0,15,0,20,1,0,25,25,125.0
4,0de3a88a-64e0-4d99-a272-8141d961e804,CUST1000,58,M,Married,Salaried,>1L,113,100000,11212.97,...,0,25.0,15,0,0,1,0,0,25,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,efe3295b-6201-4de6-b962-a96cbf100165,CUST9999,56,M,Single,Self-employed,<25K,23,150000,32661.33,...,0,25.0,15,0,0,1,0,0,25,90.0
99884,a60ed29e-94f0-4230-919a-b3e62b71053d,CUST9999,55,F,Single,Student,>1L,45,100000,41120.85,...,0,5.0,15,0,0,1,0,25,25,95.0
99885,f0f135e9-4aa4-43b9-87ee-5c2d232dd39b,CUST9999,56,M,Single,Self-employed,>1L,88,50000,44901.04,...,0,NaN,15,0,20,1,0,0,25,85.0
99886,8b981c71-f744-4e37-9061-e3677c774ac9,CUST9999,66,F,Married,Self-employed,>1L,14,100000,15550.06,...,0,15.0,15,0,0,1,0,0,0,55.0


In [44]:
df['Total_Risk_Score'].describe()

count    99888.000000
mean        85.467173
std         22.673976
min         15.000000
25%         65.000000
50%         90.000000
75%        105.000000
max        160.000000
Name: Total_Risk_Score, dtype: float64

In [43]:
df.to_csv('final_score.csv')